# You Tube transcripts

In [1]:
from utils import ChannelCrawler, ChannelAnalyzer, aggregate_analysis_files, Channel, VideoInfo

%load_ext autoreload
%autoreload 2

# Search for YT channels of Florida cities and towns

So we go over the list of all cities in Florida and search YouTube for "city of XYZ Florida" and "town of XYZ Florida". This is what `Crawler` class does. See docstring in Crawler for details.

In [2]:
crawler = ChannelCrawler()

In [3]:
crawler.start()

437it [00:27, 15.70it/s] 


For each json response we now ask ChatGPT to determine is the channel official or not, this creates csv file in analysis folder, and updates in status.csv: 

In [6]:
analyzer = ChannelAnalyzer()

In [7]:
analyzer.start()

437it [01:06,  6.53it/s]


We now aggregate the results in a excel file, storing only yes results:

In [14]:
aggregate_analysis_files(crawler, 'aggregated_analysis.xlsx')

Error reading analysis/AT00007244.csv. Skipping it.


# Get all live videos from one channel

We now feed in channel id and get all the live videos from that channel. Let's take an example of city of Belleair Beach (strangely this city is NOT in the list of cities provided):

In [13]:
# channel = Channel('UCBTiCuq7bdOfOjqAnHY0zbA')
channel = Channel('UCm9YZSpPqHckVrtDdrL3isw')


In [ ]:
channel.get_videos()

## Extract info from one video

In [18]:
video = VideoInfo("thGB9IILDOw")

In [20]:
video.get_all_video_info()

Dictionary saved to channels/UCBTiCuq7bdOfOjqAnHY0zbA/videos/thGB9IILDOw.json


# Extract all transcripts from a channel

In [22]:
channel.extract_all()

Could not get transcript for video:  3eHSnYwnX4g
Could not get transcript for video:  vSkwTgdaVN0
Could not get transcript for video:  XzvoDIGDBs8
Could not get transcript for video:  ZxWje7n_sbE
Could not get transcript for video:  kh45r3_hxog
Could not get transcript for video:  daKPMUu5hqU
Could not get transcript for video:  iJoBvmkrDlw
Could not get transcript for video:  09bcu3eOwqs
Could not get transcript for video:  wnkknvswiAk
Could not get transcript for video:  qjIV49wDChU
Could not get transcript for video:  ZUZJrksyKuA


In [23]:
video = VideoInfo('3eHSnYwnX4g')

In [24]:
video.get_only_transcript()

Could not get transcript for video:  3eHSnYwnX4g

Could not retrieve a transcript for the video https://www.youtube.com/watch?v=3eHSnYwnX4g! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
